In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

In [35]:
# convert xml to dataframe
loc = r'I:\Project\Df_merge\Piper_and_INS_xml_files\Dataframes\\'
o_loc = r'I:\Project\Df_merge\Piper_and_INS_xml_files\Output\\'
xml_loc = r'I:\Project\Df_merge\Piper_and_INS_xml_files\\'
def xml_to_df(xml_loc,xml_file):
    xml_data = open(xml_loc+xml_file+'.xml').read()
    root = ET.XML(xml_data) # element tree
    all_records = []
    for i, child in enumerate(root):
        record = {}
        for subchild in child:
            record[subchild.tag] = subchild.text
            all_records.append(record)
    return  pd.DataFrame(all_records)

In [36]:
# function to convert dataframe to csv
def conv_csv(loc,df, name):
    with open(loc +name+'.csv', 'wb') as f:
        df.to_csv(f,index = 'false')

In [37]:
trans_df = xml_to_df(xml_loc,xml_file = 'TRANSMEDIA')
equip_df = xml_to_df(xml_loc,xml_file = 'EQUIPMENT')

equip2_df = xml_to_df(xml_loc,'EQUIPMENT2')
trans2_df = xml_to_df(xml_loc,'TRANSMEDIA2')

conv_csv(loc,equip_df,'equip_df')
conv_csv(loc,equip_df,'trans_df')

In [38]:
conv_csv(loc,equip2_df,'equip2_df')

In [39]:
equip_df = equip_df[['BT_PLANT_1141_CODE','GEOMETRY','EQUIPMENT_NAME']]
equip2_df = equip2_df[['BT_PLANT_1141_CODE','GEOMETRY','EQUIPMENT_NAME']]
equip_df = equip_df.append(equip2_df)

In [40]:
conv_csv(loc,equip_df,'equip_df')

In [41]:
trans_df = trans_df[['FROM_EQUIPMENT_NAME','TO_EQUIPMENT_NAME','BT_CONDUCTOR_NUMBER','BT_CABLE_NAME','BT_CABLE_DESIGNATION','BT_CABLE_NAME_DESIGNATION','BT_CABLE_TYPE']]
trans2_df = trans2_df[['FROM_EQUIPMENT_NAME','TO_EQUIPMENT_NAME','BT_CONDUCTOR_NUMBER','BT_CABLE_NAME','BT_CABLE_DESIGNATION','BT_CABLE_NAME_DESIGNATION','BT_CABLE_TYPE']]
trans_df = trans_df.append(trans2_df)

In [42]:
# creating equipment dataframe
bt_plant = ['TNNQD','TFRFL','TFSUK','TFROY','TFCC','TALET','TEMAH','TNSST','TNNBS','TCMOJ','TFBRN','TCYFC','TAAAC','TAAAH','TAADD']

equip_final_df = equip_df[equip_df['BT_PLANT_1141_CODE'].isin(bt_plant)]

equip_final_df = equip_final_df.drop_duplicates(['BT_PLANT_1141_CODE'])

In [43]:
# creating transmedia dataframe
trans_cln_drp_df = trans_df.dropna(axis=0, how='all')
trans_cln_drp_df = trans_cln_drp_df[trans_cln_drp_df['BT_CABLE_TYPE'] != 'BFT']

trans_ddup_df = trans_cln_drp_df.drop_duplicates(['BT_CABLE_NAME'])
conv_csv(loc,trans_ddup_df,'trans_ddup_df')

In [44]:
# creating terminating df from piper
Terminating_df = trans_ddup_df[['TO_EQUIPMENT_NAME','BT_CONDUCTOR_NUMBER','BT_CABLE_NAME','BT_CABLE_DESIGNATION','BT_CABLE_NAME_DESIGNATION']]

Terminating_df = Terminating_df.rename(columns = {'TO_EQUIPMENT_NAME':'EQUIPMENT_NAME'})

In [45]:
# creating originating df from piper
Originating_df = trans_ddup_df[['FROM_EQUIPMENT_NAME','BT_CONDUCTOR_NUMBER','BT_CABLE_NAME','BT_CABLE_DESIGNATION','BT_CABLE_NAME_DESIGNATION']]

Originating_df = Originating_df.rename(columns = {'FROM_EQUIPMENT_NAME':'EQUIPMENT_NAME'})

In [46]:
# merging piper terminating cables, converting to numeric and finding the count
Equ_Ter_df = pd.merge(equip_final_df,Terminating_df,on = 'EQUIPMENT_NAME', how = 'left')
Equ_Ter_df['BT_CONDUCTOR_NUMBER'] = Equ_Ter_df['BT_CONDUCTOR_NUMBER'].convert_objects(convert_numeric=True)
Equ_Ter_count_df =Equ_Ter_df.groupby(['EQUIPMENT_NAME']).agg({'BT_CONDUCTOR_NUMBER':np.sum})
conv_csv(loc,Equ_Ter_count_df,'Equ_Ter_count_df')

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  app.launch_new_instance()


In [47]:
# merging piper originating cables, converting to numeric and finding the count 1
Equ_Org_df = pd.merge(equip_final_df,Originating_df,on = 'EQUIPMENT_NAME', how = 'left')
Equ_Org_df['BT_CONDUCTOR_NUMBER'] = Equ_Org_df['BT_CONDUCTOR_NUMBER'].convert_objects(convert_numeric=True)
Equ_Org_count_df = Equ_Org_df.groupby(['EQUIPMENT_NAME']).agg({'BT_CONDUCTOR_NUMBER':np.sum})
conv_csv(loc,Equ_Org_count_df,'Equ_Org_count_df')

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  app.launch_new_instance()


In [48]:
# creating INS station dataframe
ins_STTN_df = pd.read_csv(r"I:\Project\Df_merge\Piper_and_INS_xml_files\Location Details.csv")

ins_STTN_df = ins_STTN_df[['STTN_1141_CODE','STTN_EASTING','STTN_NORTHING']]

ins_STTN_df = ins_STTN_df.rename(columns = {'STTN_1141_CODE':'BT_PLANT_1141_CODE'})
conv_csv(loc,ins_STTN_df,'ins_STTN_df')

In [49]:
# creating INS cable dataframe originating and terminating seprately
ins_cable_df = pd.read_csv(r"I:\Project\Df_merge\Piper_and_INS_xml_files\Cable Details.csv")

ins_ter_df = ins_cable_df[['Cable Name','Subarc','Total Fibres','Terminating']]
ins_ter_df = ins_ter_df[ins_ter_df['Terminating'].isin(bt_plant)]

ins_org_df = ins_cable_df[['Cable Name','Subarc','Total Fibres','Originating']]
ins_org_df = ins_org_df[ins_org_df['Originating'].isin(bt_plant)]

conv_csv(loc,ins_ter_df,'ins_ter_df')
conv_csv(loc,ins_org_df,'ins_org_df')

In [50]:
equip_final_df['GEOMETRY_Easting'] = equip_final_df['GEOMETRY'].apply(lambda x :str(x).split('Y')[0].strip('X=').replace('"',''))

equip_final_df['GEOMETRY_Northing'] = equip_final_df['GEOMETRY'].apply(lambda x :str(x).split('Y')[1].strip('"='))

del equip_final_df['GEOMETRY']

equip_final_df['GEOMETRY_Easting'] = equip_final_df['GEOMETRY_Easting'].convert_objects(convert_numeric=True).apply(lambda x: x/10)

equip_final_df['GEOMETRY_Northing'] = equip_final_df['GEOMETRY_Northing'].convert_objects(convert_numeric=True).apply(lambda x: x/10)

equip_final_df['GEOMETRY_Easting'] = equip_final_df['GEOMETRY_Easting'].astype('O')
equip_final_df.loc[equip_final_df['GEOMETRY_Easting'].notnull(), 'GEOMETRY_Easting'] = equip_final_df.loc[equip_final_df['GEOMETRY_Easting'].notnull(), 'GEOMETRY_Easting'].astype(int)

equip_final_df['GEOMETRY_Northing'] = equip_final_df['GEOMETRY_Northing'].astype('O')
equip_final_df.loc[equip_final_df['GEOMETRY_Northing'].notnull(), 'GEOMETRY_Northing'] = equip_final_df.loc[equip_final_df['GEOMETRY_Northing'].notnull(), 'GEOMETRY_Northing'].astype(int)

conv_csv(loc,equip_final_df,'equip_final_df')

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:9: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [73]:
# merging INS station details with PIPER equipment details
ins_piper_df = pd.merge(ins_STTN_df,equip_final_df,on = 'BT_PLANT_1141_CODE', how = 'left')
# ins_piper_df = ins_piper_df.rename(columns = {'STTN_EASTING':'INS_EASTING' ,'STTN_NORTHING':'INS_NORTHING','GEOMETRY_Easting':'PIPER_EASTING','GEOMETRY_Northing':'PIPER_NORTHING'})
conv_csv(loc,ins_piper_df,'ins_piper_df')

In [52]:
# removing the decimal values in easting and northing values
ins_piper_df_geo = ins_piper_df[(np.isclose(ins_piper_df['STTN_EASTING'].values[:, None],ins_piper_df['GEOMETRY_Easting'].values.tolist(),atol = 1)) & (np.isclose(ins_piper_df['STTN_NORTHING'].values[:, None],ins_piper_df['GEOMETRY_Northing'].values.tolist(),atol = 1))]
conv_csv(loc,ins_piper_df_geo,'ins_piper_df_geo')

In [53]:
#ins conductor number

ins_ter_df = ins_ter_df.rename(columns = {'Terminating':'STTN_1141_CODE'})
ins_ter_df['Total Fibres'] = ins_ter_df['Total Fibres'].convert_objects(convert_numeric=True)
ins_ter_count_df =ins_ter_df.groupby(['STTN_1141_CODE']).agg({'Total Fibres':np.sum})


ins_org_df = ins_org_df.rename(columns = {'Originating':'STTN_1141_CODE'})
ins_org_df['Total Fibres'] = ins_org_df['Total Fibres'].convert_objects(convert_numeric=True)
ins_org_count_df =ins_org_df.groupby(['STTN_1141_CODE']).agg({'Total Fibres':np.sum})


C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:9: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [54]:
# Tnode mismatch
Tnode_mis_piper_df = ins_STTN_df[~ins_STTN_df['BT_PLANT_1141_CODE'].isin(equip_final_df['BT_PLANT_1141_CODE'].tolist())]

Tnode_mis_ins_df = equip_final_df[~equip_final_df['BT_PLANT_1141_CODE'].isin(ins_STTN_df['BT_PLANT_1141_CODE'].tolist())]

In [55]:
# geo mismatch
geo_mis_df = ins_piper_df[~ins_piper_df['BT_PLANT_1141_CODE'].isin(ins_piper_df_geo['BT_PLANT_1141_CODE'].tolist())]
conv_csv(o_loc,geo_mis_df,'geo_mis_df')

In [56]:
#### cablename and conductor no
# cable name mismatch
Equ_Org_df['Cable Name'] = Equ_Org_df['BT_CABLE_NAME']+" " +Equ_Org_df['BT_CABLE_NAME_DESIGNATION']

Equ_Ter_df['Cable Name'] = Equ_Ter_df['BT_CABLE_NAME']+" " +Equ_Ter_df['BT_CABLE_NAME_DESIGNATION']

# conductor count based on cable names - INS
ins_ter_cond_count_df =ins_ter_df.groupby(['Cable Name','Subarc','STTN_1141_CODE']).agg({'Total Fibres':np.sum})

ins_org_cond_count_df =ins_org_df.groupby(['Cable Name','Subarc','STTN_1141_CODE']).agg({'Total Fibres':np.sum})

# conductor count based on cable names - Piper
equip_final_merge_org_df = pd.merge(equip_final_df,Originating_df,on = 'EQUIPMENT_NAME',how = 'left')

equip_final_merge_Ter_df = pd.merge(equip_final_df,Terminating_df,on = 'EQUIPMENT_NAME',how = 'left')

equip_final_merge_org_df['Cable Name'] = equip_final_merge_org_df['BT_CABLE_NAME']+" " +equip_final_merge_org_df['BT_CABLE_NAME_DESIGNATION']

equip_final_merge_Ter_df['Cable Name'] = equip_final_merge_Ter_df['BT_CABLE_NAME']+" " +equip_final_merge_Ter_df['BT_CABLE_NAME_DESIGNATION']

In [57]:
conv_csv(loc,ins_ter_cond_count_df,'ins_ter_cond_count_df')
conv_csv(loc,ins_org_cond_count_df,'ins_org_cond_count_df')

conv_csv(loc,equip_final_merge_org_df,'equip_final_merge_org_df')
conv_csv(loc,equip_final_merge_Ter_df,'equip_final_merge_Ter_df')

In [58]:
# dqi_df = pd.read_csv(r"C:\Users\611346962\Downloads\TNodes Data\DQI.csv")

# dqi_ref_df = pd.DataFrame(columns = ['Tnode','Cable Designation name','INS/PIPER','No of Fibers','Designation','Terminating/Originating'])

In [59]:
# reset index
ins_ter_cond_count_df = ins_ter_cond_count_df.reset_index()
equip_final_merge_Ter_df = equip_final_merge_Ter_df.reset_index()
ins_org_cond_count_df = ins_org_cond_count_df.reset_index()
equip_final_merge_org_df = equip_final_merge_org_df.reset_index()

In [60]:
#remove index
del equip_final_merge_org_df['index']
del equip_final_merge_Ter_df['index']

In [61]:
#rearrange
equip_final_merge_org_df = equip_final_merge_org_df[['Cable Name','BT_CABLE_DESIGNATION','BT_PLANT_1141_CODE','BT_CONDUCTOR_NUMBER',]]
equip_final_merge_Ter_df = equip_final_merge_Ter_df[['Cable Name','BT_CABLE_DESIGNATION','BT_PLANT_1141_CODE','BT_CONDUCTOR_NUMBER',]]

In [62]:
# rename piper columns
equip_final_merge_org_df = equip_final_merge_org_df.rename(columns = {'BT_CABLE_DESIGNATION':'Subarc','BT_PLANT_1141_CODE':'STTN_1141_CODE','BT_CONDUCTOR_NUMBER':'Total Fibres'})
equip_final_merge_Ter_df = equip_final_merge_Ter_df.rename(columns = {'BT_CABLE_DESIGNATION':'Subarc','BT_PLANT_1141_CODE':'STTN_1141_CODE','BT_CONDUCTOR_NUMBER':'Total Fibres'})

In [63]:
# adding columns
equip_final_merge_Ter_df['INS/PIPER'] = 'PIPER'
equip_final_merge_Ter_df['Terminating/Originating'] = 'Terminating'
ins_ter_cond_count_df['INS/PIPER'] = 'INS'
ins_ter_cond_count_df['Terminating/Originating'] = 'Terminating'
equip_final_merge_org_df['INS/PIPER'] = 'PIPER'
equip_final_merge_org_df['Terminating/Originating'] = 'Originating'
ins_org_cond_count_df['INS/PIPER'] = 'INS'
ins_org_cond_count_df['Terminating/Originating'] = 'Originating'

In [64]:
# merging ins and piper df
final_ref_df = equip_final_merge_Ter_df
final_ref_df = final_ref_df.append(ins_ter_cond_count_df)
final_ref_df = final_ref_df.append(equip_final_merge_org_df)
final_ref_df = final_ref_df.append(ins_org_cond_count_df)

In [65]:
conv_csv(o_loc,final_ref_df,'final_ref_df')

In [66]:
final_ref_df2 = pd.DataFrame()
for value in bt_plant:
    final_ref_df2 = final_ref_df2.append(final_ref_df[final_ref_df['STTN_1141_CODE']== value])

In [67]:
final_ref_df2 = final_ref_df2[['STTN_1141_CODE','Subarc','Cable Name','Total Fibres','INS/PIPER','Terminating/Originating']]

In [68]:
conv_csv(loc,final_ref_df2,'final_ref_df2')
